In [12]:
import torch
import torchvision
from torchvision import datasets 
import torchvision.transforms  as T
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn 
import torch.nn.functional as F


In [2]:
mnist = datasets.MNIST (
    
    root='./DATASET/',
    train=True, 
    transform=T.ToTensor() 
    
)

In [4]:
train , val , test = torch.utils.data.random_split( mnist , [0.8 , 0.1 , 0.1] )

train_loader = DataLoader(
    
    dataset= train,
    shuffle = True,
    batch_size=64
)
val_loader = DataLoader(
    
    dataset= val,
    shuffle = True,
    batch_size=64
)


In [13]:
def conlayer_K3P1(canal_in , canal_out , stride):
    return nn.Conv2d(canal_in , canal_out , stride=stride , kernel_size=3 , padding=1)

In [17]:
class bloque_residual(nn.Module):
    def __init__(self , in_channel , out_channel , stride=1 , change_size = True):
        super().__init__() 
        ## CNN
        self.conv1= conlayer_K3P1(in_channel , out_channel , stride )
        self.bn1= nn.BatchNorm2d(out_channel)
        self.conv2 = conlayer_K3P1(out_channel , out_channel , 1 )
        self.bn2= nn.BatchNorm2d(out_channel)
        # Para cambiar el map size : 
        self.change_size = change_size
        if change_size : 
            self.residual = nn.Sequential(
                
                nn.Conv2d(in_channel ,
                                    out_channel,
                                    kernel_size=1 ,
                                    stride=stride),
                nn.BatchNorm2d(out_channel)
                
            )
        
    def forward(self, x ): 
        if not self.change_size:
            residual_in= x  
        else:
            residual_in= x=self.residual(x)
    
        y = F.ReLU(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        y = residual_in + y 
        return F.relu(y)
            
        

In [18]:
# n sirve para iterar la creacion de codigo y no tener que escribir N capas a mano

class Resnet56(nn.Module):
    def __init__(self , n=9 , num_classes=10):
        super().__init__()
        
        self.conv1 = conlayer_K3P1(3,16 ,stride=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.block1 = self.crete_block(n=9 , 
                                       in_channel=16 ,
                                       out_channel=16 ,
                                       stride=1 ,
                                       change_size= False)
       
        self.block2 = self.crete_block(n=9 , in_channel=16 , 
                                       out_channel=32,
                                       stride=2,
                                       change_size=True)
        
        self.block3 = self.crete_block(n=9, in_channel=32,
                                       out_channel=64,
                                       stride=2,)
        
        self.avg_pool = nn.AdaptiveAvgPool2d
        
        self.fc = nn.Linear(64 , num_classes)
        
        
    def crete_block (self , n , in_channel , out_channel , stride , change_size=True ):
        block = [bloque_residual(in_channel , out_channel , stride , change_size )]
        for i in range(n-1):
            block.append(bloque_residual( out_channel , out_channel , stride=1 , change_size=False ))
        return nn.Sequential(*block)
    
    def forward(self , x):
        
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.block1(y)
        y = self.block2(y)
        y = self.avg_pool(self.block3(y))
        
        return self.fc(y.view(y.size(0) , -1))
        
        

In [19]:
model = Resnet56()
optimizer_resnet56 = torch.optim.SGD(model.parameters() , lr= 0.1 , momentum=0.95 , weight_decay=1e-4)